# Custom Transformer

In [1]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn import set_config; set_config(display='diagram')
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import FeatureUnion
from sklearn.linear_model import Ridge

👇 Consider the following dataset

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np

data = pd.read_csv("data.csv")

data


,customer_state,seller_state,product_weight_g,product_length_cm,product_height_cm,product_width_cm,days_until_delivery
0,RJ,SP,1825,53,10,40,9
1,RJ,SP,700,65,18,28,9
2,RJ,SP,1825,53,10,40,11
3,RJ,SP,1825,53,10,40,12
4,RJ,SP,1825,53,10,40,14
...,...,...,...,...,...,...,...
995,RJ,SP,16800,63,58,24,30
996,RS,SP,675,40,18,35,19
997,RJ,SP,2050,45,15,35,32
998,BA,SP,8750,52,51,17,33


Each observation of the dataset represents an item being delivered from a  `seller_state` to a `customer_state`. The columns describe the size and weight of each item. The target is the number of days between the order and the delivery.

👇 In a pipeline:

- Engineer a 'volume' feature from the dimensions features
- Preserve the original product dimensions features for training
- Scale all numerical features
- Encode the categorical features
- Train a default `Ridge` regression and cross_validate its score on the train set. Low r2 score are expected.

Use your pipeline to predict the delivery of the following order

In [3]:
def vol(df):
    return pd.DataFrame(df['product_length_cm'] * df['product_height_cm']* df['product_width_cm'])

In [4]:
#volume = FunctionTransformer(volume)

In [5]:
y = data['days_until_delivery']
X = data.drop(columns=['days_until_delivery'])

In [6]:
num_transformer = Pipeline([
    ('scaler', MinMaxScaler())
])

In [7]:
cat_transformer = OneHotEncoder(handle_unknown='ignore',sparse=False)

In [8]:
preprocessor = ColumnTransformer([
    ('num_transformer', num_transformer, ['product_weight_g','product_height_cm', 'product_length_cm', 'product_width_cm']),
    ('cat_transformer', cat_transformer, ['customer_state', 'seller_state'])])

In [9]:
preprocessor

ColumnTransformer(transformers=[('num_transformer',
                                 Pipeline(steps=[('scaler', MinMaxScaler())]),
                                 ['product_weight_g', 'product_height_cm',
                                  'product_length_cm', 'product_width_cm']),
                                ('cat_transformer',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse=False),
                                 ['customer_state', 'seller_state'])])

In [10]:
preprocess_volume = Pipeline([
    ('vol_transformer', FunctionTransformer(vol)), 
    ('scaler_vol', MinMaxScaler())
    
])

In [11]:
preprocess_volume

Pipeline(steps=[('vol_transformer',
                 FunctionTransformer(func=<function vol at 0x12752cee0>)),
                ('scaler_vol', MinMaxScaler())])

In [12]:
final_preprocessor = FeatureUnion([
    ('preprocess', preprocessor),
    ('volume_preprocess', preprocess_volume)
])

In [13]:
final_preprocessor


FeatureUnion(transformer_list=[('preprocess',
                                ColumnTransformer(transformers=[('num_transformer',
                                                                 Pipeline(steps=[('scaler',
                                                                                  MinMaxScaler())]),
                                                                 ['product_weight_g',
                                                                  'product_height_cm',
                                                                  'product_length_cm',
                                                                  'product_width_cm']),
                                                                ('cat_transformer',
                                                                 OneHotEncoder(handle_unknown='ignore',
                                                                               sparse=False),
                                                                 ['customer_state',
                                                                  'seller_state'])])),
                               ('volume_preprocess',
                                Pipeline(steps=[('vol_transformer',
                                                 FunctionTransformer(func=<function vol at 0x12752cee0>)),
                                                ('scaler_vol',
                                                 MinMaxScaler())]))])

In [14]:
pd.DataFrame(final_preprocessor.fit_transform(X)).head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.057692,0.056818,0.402439,0.271028,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.079177
1,0.020067,0.147727,0.548780,0.158879,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.125218
2,0.057692,0.056818,0.402439,0.271028,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.079177
3,0.057692,0.056818,0.402439,0.271028,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.079177
4,0.057692,0.056818,0.402439,0.271028,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.079177


In [15]:
final_pipe= Pipeline([
    ('preprocessing', final_preprocessor),
    ('ridge_regression', Ridge())
    
])

In [16]:
final_pipe

Pipeline(steps=[('preprocessing',
                 FeatureUnion(transformer_list=[('preprocess',
                                                 ColumnTransformer(transformers=[('num_transformer',
                                                                                  Pipeline(steps=[('scaler',
                                                                                                   MinMaxScaler())]),
                                                                                  ['product_weight_g',
                                                                                   'product_height_cm',
                                                                                   'product_length_cm',
                                                                                   'product_width_cm']),
                                                                                 ('cat_transformer',
                                                                                  OneHotEncoder(handle_unknown='ignore',
                                                                                                sparse=False),
                                                                                  ['customer_state',
                                                                                   'seller_state'])])),
                                                ('volume_preprocess',
                                                 Pipeline(steps=[('vol_transformer',
                                                                  FunctionTransformer(func=<function vol at 0x12752cee0>)),
                                                                 ('scaler_vol',
                                                                  MinMaxScaler())]))])),
                ('ridge_regression', Ridge())])

In [21]:
from sklearn.model_selection import cross_validate


In [20]:
cv_results = cross_validate(final_pipe, X, y, cv=5, 
                            scoring=['r2']
                           )

In [22]:
cv_results['test_r2'].mean()

0.12573655053079716

In [24]:
final_pipe.fit(X, y)

Pipeline(steps=[('preprocessing',
                 FeatureUnion(transformer_list=[('preprocess',
                                                 ColumnTransformer(transformers=[('num_transformer',
                                                                                  Pipeline(steps=[('scaler',
                                                                                                   MinMaxScaler())]),
                                                                                  ['product_weight_g',
                                                                                   'product_height_cm',
                                                                                   'product_length_cm',
                                                                                   'product_width_cm']),
                                                                                 ('cat_transformer',
                                                                                  OneHotEncoder(handle_unknown='ignore',
                                                                                                sparse=False),
                                                                                  ['customer_state',
                                                                                   'seller_state'])])),
                                                ('volume_preprocess',
                                                 Pipeline(steps=[('vol_transformer',
                                                                  FunctionTransformer(func=<function vol at 0x12752cee0>)),
                                                                 ('scaler_vol',
                                                                  MinMaxScaler())]))])),
                ('ridge_regression', Ridge())])

In [17]:
new_data = pd.read_csv("data_new.csv")
new_data

,customer_state,seller_state,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,RJ,SP,1825,53,10,40


<details><summary>Hints</summary>

- There are many ways to create your preprocessed matrix (using `ColumnTransformer` and/or `FeatureUnion`). 
    
- If your transformed feature matrix look wierd, it may be stored as "sparse" by the default behavior of `OneHotEncoder(sparse=True)`. Use `.todense()` to turn it back to a dense matrix

</details>

In [26]:
X_new = new_data

In [29]:
y_new = final_pipe.predict(X_new)[0]
y_new

20.67221181581968